In [ ]:
import torch
import torchvision
from torchvision import datasets,transforms, models
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable 
import time
%matplotlib inline

In [ ]:
path = "./data"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
                             # transforms.Lambda(shear),
                             transforms.RandomSizedCrop(224),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),
                             normalize,
                                 ])

data_image = {x:datasets.ImageFolder(root = os.path.join(path,x),
                                     transform = transform)
              for x in ["train", "val"]}

data_loader_image = {x:torch.utils.data.DataLoader(dataset=data_image[x],
                                                num_workers=4,
                                                batch_size=8,
                                                pin_memory=True,
                                                shuffle = True)
                     for x in ["train", "val"]}

In [ ]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

In [ ]:
classes = data_image["train"].classes
classes_index = data_image["train"].class_to_idx
print(classes)
print(classes_index)

In [ ]:
print(u"训练集个数:", len(data_image["train"]))
print(u"验证集个数:", len(data_image["val"]))

In [ ]:
X_train, y_train = next(iter(data_loader_image["train"]))
mean = [0.5,0.5,0.5]
std  = [0.5,0.5,0.5]
img = torchvision.utils.make_grid(X_train)
img = img.numpy().transpose((1,2,0))
img = img*std+mean

print([classes[i] for i in y_train])
plt.imshow(img)

In [115]:
model = models.vgg16(pretrained=True)

In [116]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 

In [119]:
lr = 1e-2
for parma in model.parameters():
    parma.requires_grad = False

model.classifier = torch.nn.Sequential(torch.nn.Linear(25088, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 1024),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(1024, 128),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(128, 32),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(32, 8),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(8, 2),
                                      )

for index, parma in enumerate(model.classifier.parameters()):
    if index == 6:
        parma.requires_grad = True
    
if use_gpu:
    model = model.cuda()


cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(),lr, weight_decay=1e-4)

In [120]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 

In [122]:
n_epochs = 5
for epoch in range(n_epochs):
    since = time.time()
    print("Epoch{}/{}".format(epoch, n_epochs))
    print("-"*10)
    for param in ["train", "val"]:
        if param == "train":
            model.train = True
            model = torch.nn.DataParallel(model)
        else:
            model.train = False

        running_loss = 0.0
        running_correct = 0 
        batch = 0
        for data in data_loader_image[param]:
            batch += 1
            X, y = data
            if use_gpu:
                X, y  = Variable(X.cuda()), Variable(y.cuda())
            else:
                X, y = Variable(X), Variable(y)
        
            optimizer.zero_grad()
            y_pred = model(X)
            _, pred = torch.max(y_pred.data, 1)
        
            loss = cost(y_pred, y)
            if param =="train":
                loss.backward()
                optimizer.step()
            running_loss += loss.data[0]
            running_correct += torch.sum(pred == y.data)
            if batch%500 == 0 and param =="train":
                print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}".format(
                      batch, running_loss/(4*batch), 100*running_correct/(4*batch)))
            
        epoch_loss = running_loss/len(data_image[param])
        epoch_correct = 100*running_correct/len(data_image[param])

        print("{}  Loss: {:.4f},  Correct: {:.4f}".format(param, epoch_loss, epoch_correct))
    now_time = time.time() - since   
    print("Training time is:{:.0f}m {:.0f}s".format(now_time//60, now_time%60))

Epoch0/2
----------
Batch 500, Train Loss:0.2011, Train ACC:100.9500
Batch 1000, Train Loss:0.1873, Train ACC:100.5500
Batch 1500, Train Loss:0.1842, Train ACC:99.1667
Batch 2000, Train Loss:0.1826, Train ACC:99.7875
Batch 2500, Train Loss:0.1808, Train ACC:99.8900
train  Loss: 0.0900,  Correct: 49.8756
val  Loss: 0.0870,  Correct: 50.0000
Training time is:2m 0s
Epoch1/2
----------
Batch 500, Train Loss:0.1737, Train ACC:95.0000
Batch 1000, Train Loss:0.1735, Train ACC:98.5250
Batch 1500, Train Loss:0.1735, Train ACC:98.6333
Batch 2000, Train Loss:0.1735, Train ACC:99.3000
Batch 2500, Train Loss:0.1735, Train ACC:99.8100
train  Loss: 0.0867,  Correct: 50.1244
val  Loss: 0.0868,  Correct: 50.0000
Training time is:2m 1s


In [123]:
torch.save(model.state_dict(), "model_vgg16_finetune.pkl")

In [ ]:
model.load_state_dict(torch.load('model_vgg16_finetune.pkl'))

In [ ]:
dir(torch)

In [124]:
class ImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, path

In [125]:
data_test_img = ImageFolder(root="./test/",
                                     transform = transform)
data_loader_test_img = torch.utils.data.DataLoader(dataset=data_test_img,
                                                   batch_size = 1)

In [126]:
for a,b,c in data_loader_test_img:
    print(a)
    print(b)
    print(c)
    break


( 0 , 0 ,.,.) = 
 -1.5014 -1.5357 -1.5870  ...   1.2728  1.2899  1.3242
 -1.5528 -1.5870 -1.6384  ...   1.2899  1.2728  1.2728
 -1.6042 -1.6384 -1.6898  ...   1.3584  1.3242  1.3070
           ...             ⋱             ...          
  0.3994  0.2453  0.0227  ...   0.8789  0.8618  0.8447
  0.4508  0.4508  0.2967  ...   0.8447  0.8276  0.8104
  1.0502  1.0502  0.6563  ...   0.8104  0.7933  0.7762

( 0 , 1 ,.,.) = 
 -1.2304 -1.2654 -1.3354  ...   0.3803  0.3978  0.4328
 -1.2654 -1.3179 -1.3704  ...   0.3978  0.3803  0.3803
 -1.3179 -1.3529 -1.4230  ...   0.4678  0.4328  0.4153
           ...             ⋱             ...          
  0.3102  0.1702 -0.0224  ...   1.1856  1.1506  1.1506
  0.3452  0.3627  0.2227  ...   1.1506  1.1155  1.1331
  0.9755  0.9755  0.5903  ...   1.0980  1.0805  1.0980

( 0 , 2 ,.,.) = 
 -0.8458 -0.8981 -0.9853  ...   0.4265  0.4439  0.4788
 -0.8981 -0.9504 -1.0376  ...   0.4439  0.4265  0.4265
 -0.9504 -1.0201 -1.0898  ...   0.4962  0.4614  0.4614
           

In [127]:
import collections
clip = 0.001
cnt = 1
csv_map = {}

for image, label, path in (data_loader_test_img):
    images = Variable(image.cuda())
    y_pred = model(images)
    smax = torch.nn.Softmax()
    smax_out = smax(y_pred)[0]
    cat_prob = smax_out.data[0]
    dog_prob = smax_out.data[1]
    prob = dog_prob

    if cat_prob > dog_prob:
        prob = 1 - cat_prob
    prob = np.around(prob, decimals=4)
    prob = np.clip(prob, clip, 1-clip)
    filepath = path[0].split('/')[-1].split('.')[-2]
    if filepath == '0':
        print(path)
        break
    csv_map[filepath] = prob
    


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


In [128]:
csv_list = []
for k,v in csv_map.items():
    csv_list.append((k,v))

In [129]:
len(csv_list)

12500

In [130]:
import pandas as pd

In [131]:
df = pd.DataFrame(csv_list)
df.columns = ['id','label']
df.id = df.id.astype(int)
df = df.sort_values('id')

In [132]:
df.to_csv('result.csv',index=False)

In [ ]:
df.groupby('id').count()

In [ ]:
img = torchvision.utils.make_grid(image)
img = img.numpy().transpose(1,2,0)
mean = [0.5,0.5,0.5]
std  = [0.5,0.5,0.5]
img = img*std+mean
print("Pred Label:",[classes[i] for i in pred])
plt.imshow(img)